In [118]:
import pickle
import pandas as pd
import numpy as np

df = pickle.load(open('dataset_2.pkl', 'rb'))

In [119]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,West Indies,India,0.1,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground
1,2,West Indies,India,0.2,0,JD Campbell,Lauderhill,Central Broward Regional Park Stadium Turf Ground
2,2,West Indies,India,0.3,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground
3,2,West Indies,India,0.4,1,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground
4,2,West Indies,India,0.5,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground


Feature extraction and which features we need

1. batting_team
2. bowling_team
3. city
4. current_score
5. balls_left
6. wickets_left
7. current_run_rate
8. last_five_over_runs

In [120]:
df.isnull().sum()

match_id               0
batting_team           0
bowling_team           0
ball                   0
runs                   0
player_dismissed       0
city                8545
venue                  0
dtype: int64

## creating city column

In [121]:
df['city'].isna().sum()

8545

In [122]:
df['city'].unique()

array(['Lauderhill', 'St Lucia', 'Bangalore', 'Nottingham', 'Cape Town',
       'Wellington', nan, 'Hamilton', 'Chandigarh', 'Colombo',
       'Southampton', 'Trinidad', 'Durban', 'Dhaka', 'Mirpur', 'Rajkot',
       'Auckland', 'Johannesburg', 'Pune', 'Kolkata', 'Nagpur', 'Bristol',
       'London', 'Kanpur', 'Dehradun', 'Hyderabad', 'Barbados', 'Mumbai',
       'Sydney', 'Lahore', 'Karachi', 'Mount Maunganui', 'Cardiff',
       'Christchurch', 'Abu Dhabi', 'St Vincent', 'Chennai', 'Manchester',
       'Ranchi', 'Thiruvananthapuram', 'Delhi', 'Hobart',
       'Chester-le-Street', 'St Kitts', 'Napier', 'Port Elizabeth',
       'Perth', 'Bengaluru', 'Cuttack', 'Gros Islet', 'Brisbane',
       'Centurion', 'Chittagong', 'Guyana', 'Birmingham', 'Kandy',
       'Lucknow', 'Visakhapatnam', 'Potchefstroom', 'Jamaica',
       'Hambantota', 'East London', 'Canberra', 'Paarl', 'Victoria',
       'Adelaide', 'Chattogram', 'Basseterre', 'Dharmasala', 'Dominica',
       'Indore', 'Melbourne', 'Ahme

In [123]:
df[df['city'].isnull()]['venue'].value_counts()

Dubai International Cricket Stadium        2966
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: venue, dtype: int64

In [124]:
# np.where(question, if true fill, if false fill)
cities = np.where(df['city'].isnull(), df['venue'].str.split().apply(lambda x:x[0]), df['city'])

In [125]:
df['city'] = cities

In [126]:
df['city'].isnull().sum()

0

In [127]:
df.drop(columns=['venue'], inplace = True)

In [128]:
cites_more_than_600_balls = df['city'].value_counts()>600

In [129]:
eligible_city = df['city'].value_counts()[cites_more_than_600_balls].index.tolist()

In [130]:
df = df[df['city'].isin(eligible_city)]

In [131]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,West Indies,India,0.1,0,0,Lauderhill
1,2,West Indies,India,0.2,0,JD Campbell,Lauderhill
2,2,West Indies,India,0.3,0,0,Lauderhill
3,2,West Indies,India,0.4,1,0,Lauderhill
4,2,West Indies,India,0.5,0,0,Lauderhill


## Creating current_score column

In [132]:
df['current_score'] = df.groupby('match_id').cumsum()['runs']

In [133]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score
0,2,West Indies,India,0.1,0,0,Lauderhill,0
1,2,West Indies,India,0.2,0,JD Campbell,Lauderhill,0
2,2,West Indies,India,0.3,0,0,Lauderhill,0
3,2,West Indies,India,0.4,1,0,Lauderhill,1
4,2,West Indies,India,0.5,0,0,Lauderhill,1
...,...,...,...,...,...,...,...,...
118,964,Pakistan,New Zealand,19.2,1,0,Auckland,149
119,964,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Auckland,149
120,964,Pakistan,New Zealand,19.4,2,0,Auckland,151
121,964,Pakistan,New Zealand,19.5,1,0,Auckland,152


## Creating bowls_left column

In [134]:
df['over'] = df['ball'].apply(lambda x : str(x).split('.')[0])
df['ball_number'] = df['ball'].apply(lambda x : str(x).split('.')[1])

In [135]:
df['balls_bowled'] = (df['over'].astype(int)*6) + (df['ball_number'].astype(int))

In [136]:
df.tail()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_number,balls_bowled
118,964,Pakistan,New Zealand,19.2,1,0,Auckland,149,19,2,116
119,964,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Auckland,149,19,3,117
120,964,Pakistan,New Zealand,19.4,2,0,Auckland,151,19,4,118
121,964,Pakistan,New Zealand,19.5,1,0,Auckland,152,19,5,119
122,964,Pakistan,New Zealand,19.6,1,0,Auckland,153,19,6,120


In [137]:
df['balls_left'] = 120 - df['balls_bowled']

In [138]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_number,balls_bowled,balls_left
0,2,West Indies,India,0.1,0,0,Lauderhill,0,0,1,1,119
1,2,West Indies,India,0.2,0,JD Campbell,Lauderhill,0,0,2,2,118
2,2,West Indies,India,0.3,0,0,Lauderhill,0,0,3,3,117
3,2,West Indies,India,0.4,1,0,Lauderhill,1,0,4,4,116
4,2,West Indies,India,0.5,0,0,Lauderhill,1,0,5,5,115


## Creating wickets_left colum

In [139]:
df['player_dismissed'] = df['player_dismissed'].apply(lambda x: 0 if x == '0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype(int)

In [140]:
df['player_dismissed'] = df.groupby('match_id').cumsum()['player_dismissed']

In [141]:
df['wickets_left'] = 10 - df['player_dismissed']

In [142]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_number,balls_bowled,balls_left,wickets_left
0,2,West Indies,India,0.1,0,0,Lauderhill,0,0,1,1,119,10
1,2,West Indies,India,0.2,0,1,Lauderhill,0,0,2,2,118,9
2,2,West Indies,India,0.3,0,1,Lauderhill,0,0,3,3,117,9
3,2,West Indies,India,0.4,1,1,Lauderhill,1,0,4,4,116,9
4,2,West Indies,India,0.5,0,1,Lauderhill,1,0,5,5,115,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,964,Pakistan,New Zealand,19.2,1,8,Auckland,149,19,2,116,4,2
119,964,Pakistan,New Zealand,19.3,0,9,Auckland,149,19,3,117,3,1
120,964,Pakistan,New Zealand,19.4,2,9,Auckland,151,19,4,118,2,1
121,964,Pakistan,New Zealand,19.5,1,9,Auckland,152,19,5,119,1,1


## create Current run rate

current_score/num_of_balls

In [143]:
df['current_run_rate'] = (df['current_score']*6)/df['balls_bowled']

In [144]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_number,balls_bowled,balls_left,wickets_left,current_run_rate
0,2,West Indies,India,0.1,0,0,Lauderhill,0,0,1,1,119,10,0.000000
1,2,West Indies,India,0.2,0,1,Lauderhill,0,0,2,2,118,9,0.000000
2,2,West Indies,India,0.3,0,1,Lauderhill,0,0,3,3,117,9,0.000000
3,2,West Indies,India,0.4,1,1,Lauderhill,1,0,4,4,116,9,1.500000
4,2,West Indies,India,0.5,0,1,Lauderhill,1,0,5,5,115,9,1.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,964,Pakistan,New Zealand,19.2,1,8,Auckland,149,19,2,116,4,2,7.706897
119,964,Pakistan,New Zealand,19.3,0,9,Auckland,149,19,3,117,3,1,7.641026
120,964,Pakistan,New Zealand,19.4,2,9,Auckland,151,19,4,118,2,1,7.677966
121,964,Pakistan,New Zealand,19.5,1,9,Auckland,152,19,5,119,1,1,7.663866


## last_five_over_runs

In [145]:
groups = df.groupby('match_id')

match_ids = df['match_id'].unique()
last_five = []

for i in match_ids:
    #sum of runs for last five overs (30 balls- 5 overs)
    list_of_runs_each_group = groups.get_group(i).rolling(window = 30).sum()['runs'].values.tolist()
    last_five.extend(list_of_runs_each_group)

/var/folders/2m/bcrsq2rn60n_0qbc80sw7mqm0000gn/T/ipykernel_90740/3420672983.py:8: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['batting_team', 'bowling_team', 'city'], dtype='object')
  list_of_runs_each_group = groups.get_group(i).rolling(window = 30).sum()['runs'].values.tolist()


In [146]:
df['last_five_over_runs'] = last_five

1. batting_team
2. bowling_team
3. city
4. current_score
5. balls_left
6. wickets_left
7. current_run_rate
8. last_five_over_runs

In [147]:
# df[['batting_team', 'bowling_team', 'city', 'current_score', 'balls_left', 'wickets_left', 'current_run_rate', 'last_five_over_runs']]

## Final_score column

In [148]:
final_df = df.groupby('match_id').sum()['runs'].reset_index().merge(df, on = 'match_id')

In [149]:
final_df

,match_id,runs_x,batting_team,bowling_team,ball,runs_y,player_dismissed,city,current_score,over,ball_number,balls_bowled,balls_left,wickets_left,current_run_rate,last_five_over_runs
0,2,95,West Indies,India,0.1,0,0,Lauderhill,0,0,1,1,119,10,0.000000,NaN
1,2,95,West Indies,India,0.2,0,1,Lauderhill,0,0,2,2,118,9,0.000000,NaN
2,2,95,West Indies,India,0.3,0,1,Lauderhill,0,0,3,3,117,9,0.000000,NaN
3,2,95,West Indies,India,0.4,1,1,Lauderhill,1,0,4,4,116,9,1.500000,NaN
4,2,95,West Indies,India,0.5,0,1,Lauderhill,1,0,5,5,115,9,1.200000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50857,964,153,Pakistan,New Zealand,19.2,1,8,Auckland,149,19,2,116,4,2,7.706897,56.0
50858,964,153,Pakistan,New Zealand,19.3,0,9,Auckland,149,19,3,117,3,1,7.641026,50.0
50859,964,153,Pakistan,New Zealand,19.4,2,9,Auckland,151,19,4,118,2,1,7.677966,52.0
50860,964,153,Pakistan,New Zealand,19.5,1,9,Auckland,152,19,5,119,1,1,7.663866,53.0


In [150]:
 final_df = final_df[['batting_team', 'bowling_team', 'city', 'current_score', 'balls_left', 'wickets_left', 'current_run_rate', 'last_five_over_runs', 'runs_x', 'runs_y']]

In [152]:
final_df.drop(columns='runs_y', inplace=True)

/var/folders/2m/bcrsq2rn60n_0qbc80sw7mqm0000gn/T/ipykernel_90740/2079763568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.drop(columns='runs_y', inplace=True)


In [154]:
final_df['final_score'] = final_df['runs_x']

/var/folders/2m/bcrsq2rn60n_0qbc80sw7mqm0000gn/T/ipykernel_90740/2917466074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['final_score'] = final_df['runs_x']


In [156]:
final_df.drop(columns='runs_x', inplace=True)

/var/folders/2m/bcrsq2rn60n_0qbc80sw7mqm0000gn/T/ipykernel_90740/99266869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.drop(columns='runs_x', inplace=True)


In [157]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,current_run_rate,last_five_over_runs,final_score
0,West Indies,India,Lauderhill,0,119,10,0.000000,NaN,95
1,West Indies,India,Lauderhill,0,118,9,0.000000,NaN,95
2,West Indies,India,Lauderhill,0,117,9,0.000000,NaN,95
3,West Indies,India,Lauderhill,1,116,9,1.500000,NaN,95
4,West Indies,India,Lauderhill,1,115,9,1.200000,NaN,95
...,...,...,...,...,...,...,...,...,...
50857,Pakistan,New Zealand,Auckland,149,4,2,7.706897,56.0,153
50858,Pakistan,New Zealand,Auckland,149,3,1,7.641026,50.0,153
50859,Pakistan,New Zealand,Auckland,151,2,1,7.677966,52.0,153
50860,Pakistan,New Zealand,Auckland,152,1,1,7.663866,53.0,153


In [158]:
final_df.dropna(inplace=True)

/var/folders/2m/bcrsq2rn60n_0qbc80sw7mqm0000gn/T/ipykernel_90740/2709626079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.dropna(inplace=True)


In [162]:
rows = final_df.shape[0]
final_df = final_df.sample(rows)

In [163]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,current_run_rate,last_five_over_runs,final_score
44165,India,England,Colombo,168,-2,6,8.262295,51.0,170
36002,Australia,India,Melbourne,74,42,5,5.692308,26.0,132
44362,South Africa,India,Mirpur,81,51,8,7.043478,34.0,172
4236,Sri Lanka,England,St Lucia,40,74,7,5.217391,21.0,128
28512,Bangladesh,India,Colombo,110,30,5,7.333333,42.0,166
...,...,...,...,...,...,...,...,...,...
30900,Pakistan,Australia,Abu Dhabi,110,34,8,7.674419,37.0,155
4361,Pakistan,Australia,Colombo,38,77,8,5.302326,30.0,149
47544,West Indies,India,Lauderhill,98,76,10,13.363636,72.0,245
24624,Australia,Pakistan,Johannesburg,36,93,8,8.000000,36.0,164


In [164]:
final_df.to_csv('final_df.csv')